In [1]:
from dotenv import load_dotenv
load_dotenv()
import os
google_api_key = os.getenv('GOOGLE_API_KEY')
if not google_api_key: 
    raise ValueError("GOOGLE_API_KEY is not set in the environment variables.")

In [2]:
from langchain.chat_models import init_chat_model
llm = init_chat_model("gemini-2.0-flash", model_provider="google_genai")

In [3]:
from langgraph.prebuilt import create_react_agent

def get_weather(city: str) -> str:  
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

agent = create_react_agent(
    model=llm,  
    tools=[get_weather],  
    prompt="You are a helpful assistant"  
)

# Run the agent
agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in San Francisco?"}]}
)

{'messages': [HumanMessage(content='what is the weather in San Francisco?', additional_kwargs={}, response_metadata={}, id='e0fc366b-3880-4a4f-8716-81a2f932809f'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_weather', 'arguments': '{"city": "San Francisco"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--9eb02329-5edc-41f0-8b9e-dc70275e81f9-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'San Francisco'}, 'id': 'f1239a8e-54ac-464f-b2a0-d14de7164a8b', 'type': 'tool_call'}], usage_metadata={'input_tokens': 27, 'output_tokens': 6, 'total_tokens': 33, 'input_token_details': {'cache_read': 0}}),
  ToolMessage(content="It's always sunny in San Francisco!", name='get_weather', id='609da351-92a0-427a-ac65-b0159f3ce64e', tool_call_id='f1239a8e-54ac-464f-b2a0-d14de7164a8b'),
  AIMessage(content="It's always sunny in San Francisco!"

In [4]:
agent = create_react_agent(
    model=llm,  
    tools=[get_weather],  
    prompt="Never answer questions about the weather."  
)

# Run the agent
agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in San Francisco?"}]}
)

{'messages': [HumanMessage(content='what is the weather in San Francisco?', additional_kwargs={}, response_metadata={}, id='217a017c-ae51-4160-ad92-0288a4ee24fb'),
  AIMessage(content='I am sorry, I cannot fulfill this request. The available tools lack the ability to provide weather information.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--e2c66846-3a1e-4b74-82e4-512553b8a438-0', usage_metadata={'input_tokens': 29, 'output_tokens': 22, 'total_tokens': 51, 'input_token_details': {'cache_read': 0}})]}

In [5]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import InMemorySaver

checkpointer = InMemorySaver()

agent = create_react_agent(
    model=llm,
    tools=[get_weather],
    checkpointer=checkpointer  
)

# Run the agent
config = {"configurable": {"thread_id": "1"}}
sf_response = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in san francisco?"}]},
    config  
)
ny_response = agent.invoke(
    {"messages": [{"role": "user", "content": "what about new york?"}]},
    config
)

In [6]:
sf_response, ny_response

({'messages': [HumanMessage(content='what is the weather in san francisco?', additional_kwargs={}, response_metadata={}, id='31a41ef8-f3cd-4adf-b1ce-69d63dffbd9b'),
   AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_weather', 'arguments': '{"city": "san francisco"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--4c414b2c-602e-466e-aa57-31c0ad75d4c8-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'san francisco'}, 'id': '8f823329-e325-4a98-b2b1-a2d25f5a4844', 'type': 'tool_call'}], usage_metadata={'input_tokens': 22, 'output_tokens': 6, 'total_tokens': 28, 'input_token_details': {'cache_read': 0}}),
   ToolMessage(content="It's always sunny in san francisco!", name='get_weather', id='59b0093c-3ca7-4a30-bc67-7df18c6d639f', tool_call_id='8f823329-e325-4a98-b2b1-a2d25f5a4844'),
   AIMessage(content="It's always sunny in San Francis

In [7]:
from pydantic import BaseModel

class WeatherResponse(BaseModel):
    city: str
    conditions: str

agent = create_react_agent(
    model=llm,
    tools=[get_weather],
    response_format=WeatherResponse  
)
response = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in san francisco?"}]}
)

response["structured_response"]

WeatherResponse(city='San Francisco', conditions='Sunny')